In [1]:
import pandas as pd

In [2]:
# Paso 1: Leer informe Excel Clientes
f1 = pd.DataFrame(columns=['CEDULA', 'Nombre y apellido'])
#f1 = pd.read_excel("archivos2/BASE SOCIAL A 31 DE DICIEMBRE DE 2023 APORTES Y CREDITOS.xlsx")
#f1 = pd.read_excel("archivos/BASE DE DATOS AÑO 2023 INGRESOS Y EGRESOS.xlsx")
f1.columns = [i.strip() for i in f1.columns] # Removiendo espacios en columnas
f1 = f1.rename(columns={"NIT":"CEDULA", "Cedula":"CEDULA", "NOMBREINTE":"Nombre y apellido", "AGENCIA":"COD AGENCIA"})
f1["CEDULA"] = f1["CEDULA"].astype('int64')  # Convirtiendo cédula a entero
f1

,CEDULA,Nombre y apellido


In [3]:
# Paso 2: Leer informe Excel Transacciones
f2 = pd.read_csv("ARCHIVOS4/Transaciones febrero.csv", delimiter=";")
#f2 = pd.read_excel("ARCHIVOS3/TRANSACCIONES HISTORICOS 2023.xlsx")
#f2 = pd.read_excel("archivos/BD VISIONAMOS.xlsx")
#f2 = pd.read_excel("archivos2/TRANSACCIONES VISIONAMOS.xlsx")
f2

,Fecha,Cedula,documento,JURISDICCION,totalefectivo,codigocuenta,Tipo de movimiento,Descripción,CANAL
0,1/02/2024,54257726,01 12720,1,619.0,27400516,C,PAGO DE CUOTA DE CREDITO,VENTANILLA
1,1/02/2024,54257726,01 12720,1,33334.0,31050505,C,PAGO DE CUOTA DE CREDITO,VENTANILLA
2,1/02/2024,54257726,01 12720,1,11134.0,16551801,C,PAGO DE CUOTA DE CREDITO,VENTANILLA
3,1/02/2024,54257726,01 12720,1,179913.0,14410501,C,PAGO DE CUOTA DE CREDITO,VENTANILLA
4,1/02/2024,54257726,01 12720,1,225000.0,11050501,C,PAGO DE CUOTA DE CREDITO,VENTANILLA
...,...,...,...,...,...,...,...,...,...
6684,11/03/2024,1128427330,01 12810,1,100000.0,11050501,C,CONSIGNACION AHORRO ASAMBLEA,VENTANILLA
6685,11/03/2024,1036673035,01 12810,1,100000.0,21050505,C,CONSIGNACION AHORRO ASAMBLEA,VENTANILLA
6686,11/03/2024,1036673035,01 12810,1,100000.0,11050501,C,CONSIGNACION AHORRO ASAMBLEA,VENTANILLA
6687,11/03/2024,1077435947,02 12398,1,466250579.9,11050501,R,RETIRO FINAL DEL CAJA,VENTANILLA


In [4]:
f2.columns = [i.strip() for i in f2.columns] # Removiendo espacios en columnas
f2.columns = [col.upper() for col in f2.columns]
f2 = f2.rename(columns={"CEDULA":"DOCUMENTO1", "TOTAL EFECTIVO":"VALOR", "TIPO DE MOVIMIENTO":"OPERACION",
                       "AGENCIA": "COD. AGENCIA", "TIPODEMOVIMIENTO":"OPERACION", "TOTALEFECTIVO": "VALOR",
                       "JURISDICCION": "COD. AGENCIA"})

f2["OPERACION"] = f2["OPERACION"].apply(lambda x: "CREDITO" if "C" in x else "DEBITO")
if 'DISPOSITIVO' not in f2.columns: f2['DISPOSITIVO']=''
if 'COD. AGENCIA' not in f2.columns: f2["COD. AGENCIA"] = '-'
f2 = f2.rename(columns={'FECHA_REGISTRO':'FECHA'})
if "ESTADO" in f2.columns: f2 = f2[f2["ESTADO"]=="APROBADA"]
if 'FECHA_REGISTRO' in f2.columns:  
    f2["FECHA"] = pd.to_datetime(f2['FECHA'], format=formato) 
else: 
    try:
        f2["FECHA"] = pd.to_datetime(f2['FECHA'], format="%d/%m/%Y")  
    except:
        f2["FECHA"] = pd.to_datetime(f2['FECHA'])  
    
f2['Mes'] = f2['FECHA'].dt.to_period('M')  # Extrayendo el mes de la fecha
f2["DOCUMENTO1"] = f2["DOCUMENTO1"].astype('int64', errors='ignore') # Convirtiendo documento a entero
f2['CANAL'] = f2['CANAL'] + ' ' + f2['DISPOSITIVO']
condAgencia = 1 if "COD. AGENCIA" in f2.columns else 0
f2

,FECHA,DOCUMENTO1,DOCUMENTO,COD. AGENCIA,VALOR,CODIGOCUENTA,OPERACION,DESCRIPCIÓN,CANAL,DISPOSITIVO,Mes
0,2024-02-01,54257726,01 12720,1,619.0,27400516,CREDITO,PAGO DE CUOTA DE CREDITO,VENTANILLA,,2024-02
1,2024-02-01,54257726,01 12720,1,33334.0,31050505,CREDITO,PAGO DE CUOTA DE CREDITO,VENTANILLA,,2024-02
2,2024-02-01,54257726,01 12720,1,11134.0,16551801,CREDITO,PAGO DE CUOTA DE CREDITO,VENTANILLA,,2024-02
3,2024-02-01,54257726,01 12720,1,179913.0,14410501,CREDITO,PAGO DE CUOTA DE CREDITO,VENTANILLA,,2024-02
4,2024-02-01,54257726,01 12720,1,225000.0,11050501,CREDITO,PAGO DE CUOTA DE CREDITO,VENTANILLA,,2024-02
...,...,...,...,...,...,...,...,...,...,...,...
6684,2024-03-11,1128427330,01 12810,1,100000.0,11050501,CREDITO,CONSIGNACION AHORRO ASAMBLEA,VENTANILLA,,2024-03
6685,2024-03-11,1036673035,01 12810,1,100000.0,21050505,CREDITO,CONSIGNACION AHORRO ASAMBLEA,VENTANILLA,,2024-03
6686,2024-03-11,1036673035,01 12810,1,100000.0,11050501,CREDITO,CONSIGNACION AHORRO ASAMBLEA,VENTANILLA,,2024-03
6687,2024-03-11,1077435947,02 12398,1,466250579.9,11050501,DEBITO,RETIRO FINAL DEL CAJA,VENTANILLA,,2024-03


In [5]:
# Paso 3: Extrayendo datos de clientes
b1 = f1[["CEDULA", "Nombre y apellido"]].copy()
for col in ["Ingresos", "Egresos", "Activos", "Pasivos"]:
    b1[col] = f1[col].values if col in f1.columns else 0
b1["Codigo"] = 0 # Código actividad económica: CERO
b1 = b1.fillna(0)
b1.to_excel("Clientes.xlsx", index=False)
b1

,CEDULA,Nombre y apellido,Ingresos,Egresos,Activos,Pasivos,Codigo


In [6]:
# Paso 4: Extrayendo datos de transacciones
cols = ['DOCUMENTO1', 'Mes', 'OPERACION', 'CANAL', 'VALOR']
if condAgencia: cols=cols[:-1]+["COD. AGENCIA", "VALOR"]
f2 = f2[cols] # Extrayendo columnas

# Agrupando por documento, mes, operacion y canal: (Hallando conteo y suma del valor)
b2 = f2.groupby(cols[:-1]).agg({'count', 'sum'}).reset_index()
b2.columns = b2.columns.droplevel(level=0)

# Renombrando nuevamente las columnas (Por la agrupacion se borraron los nombres)
b2.columns.values[:len(cols)-1] = cols[:-1]
b2 = b2.rename(columns={"sum":"VALOR", "count":"N_VALOR"})

# Creando variable productos
b2["PRODUCTO"] = "Ahorros"
b2["DOCUMENTO1"] = b2["DOCUMENTO1"].astype('int64') # Convirtiendo documento a entero

# Agregando columna jurisdiccion (COD. AGENCIA) a los datos desde clientes
if condAgencia==0:
    b2 = pd.merge(b2, f1[['CEDULA', 'COD. AGENCIA']], left_on='DOCUMENTO1', right_on='CEDULA', how='left')
    b2 = b2.drop("CEDULA", axis=1)
b2 = b2.fillna("*")

# Renombrar columnas debito y creditos
columns = ["Mes", "DOCUMENTO1", "N_VALOR", "VALOR", "PRODUCTO", 'CANAL', "COD. AGENCIA"]
b21 = b2[b2["OPERACION"]=="DEBITO"][columns].rename(columns={"Mes":"Fecha", "DOCUMENTO1":"CEDULA", 
                                                             'CANAL':"CANAL", "N_VALOR":"N_DEBITO",
                                                             "VALOR":"SUMA_DEBITO", "COD. AGENCIA":"JURISDICCION"})

b22 = b2[b2["OPERACION"]=="CREDITO"][columns].rename(columns={"Mes":"Fecha", "DOCUMENTO1":"CEDULA", 
                                                              'CANAL':"CANAL", "N_VALOR":"N_CREDITO",
                                                              "VALOR":"SUMA_CREDITO", "COD. AGENCIA":"JURISDICCION"})


# Concatenando columnas debito, creditos y ordenando
b2 = pd.concat([b21, b22]).fillna(0).sort_values(by=["CEDULA", "Fecha"])
b2 = b2[["Fecha", "CEDULA", "N_DEBITO", "SUMA_DEBITO", "N_CREDITO", 
         "SUMA_CREDITO", "PRODUCTO", "CANAL", "JURISDICCION"]]
b2.to_excel("Transacciones.xlsx", index=False)
b2

,Fecha,CEDULA,N_DEBITO,SUMA_DEBITO,N_CREDITO,SUMA_CREDITO,PRODUCTO,CANAL,JURISDICCION
0,2024-02,1593330,0.0,0.0,6.0,400000.0,Ahorros,VENTANILLA,2
2,2024-02,3396826,4.0,30120.0,0.0,0.0,Ahorros,VENTANILLA,1
1,2024-02,3396826,0.0,0.0,6.0,1600000.0,Ahorros,VENTANILLA,1
3,2024-03,3396826,0.0,0.0,2.0,200000.0,Ahorros,VENTANILLA,1
5,2024-02,4797540,4.0,5020000.0,0.0,0.0,Ahorros,VENTANILLA,1
...,...,...,...,...,...,...,...,...,...
1059,2024-03,1214730975,0.0,0.0,4.0,100000.0,Ahorros,VENTANILLA,1
1060,2024-02,9012724244,0.0,0.0,3.0,11490000.0,Ahorros,VENTANILLA,1
1061,2024-02,9012724244,0.0,0.0,5.0,11490000.0,Ahorros,VENTANILLA,2
1062,2024-03,9012724244,0.0,0.0,3.0,72698138.0,Ahorros,VENTANILLA,1
